<a href="https://colab.research.google.com/github/leiderTic/Proyecto-Almacen/blob/main/Rnr_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalaciones

In [ ]:
#!python -m pip install statsmodels

In [ ]:
# Configuraciones de acceso

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
## Importación de librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
# Importando de keras las librerias mas importantes!
from keras.models import Sequential # Arquitectura de red neuronal!
from keras.layers import Dense      # Capa densa!
from keras.layers import LSTM       # Capa recurrente
from keras.layers import Dropout    # Evita el overfitting (Inactiva algunas neuronas)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
# Creacion de objeto datasetIngestado con atributos de los distintos dataframes del proceso

In [ ]:
class datasetIngestadoRnrLstm():
  def __init__(self, dataframe):
        self.df = dataframe
        self.df['Date'] = pd.to_datetime(self.df['Date'], dayfirst=True)
        self.df.set_index('Date', inplace=True)
        self.df.sort_index(inplace=True, ascending=True)
        self.train = pd.DataFrame()
        self.test = pd.DataFrame()
        self.model = Sequential()
        self.modelo_entrenado = 1
        self.train_max = 0.0
        self.train_min = 0.0
        self.scaler = None
        self.df_resultado = pd.DataFrame()




  def imprimir_dataset(self, n_filas):
    print(self.df.head(n_filas))

  def imprime_forma(self):
    print(self.df.shape)



  def visualiza_dataset(self, variable):
    plt.figure(num=None, figsize=(15, 6), dpi=80, facecolor='w', edgecolor='k')
    self.df[variable].plot()
    plt.tight_layout()
    plt.grid()
    plt.show()

  def descompone_serie(self, variable, n_periodos):
    res = sm.tsa.seasonal_decompose(self.df[variable],period=n_periodos)
    fig = res.plot()
    fig.set_figheight(8)
    fig.set_figwidth(15)
    plt.show()

  def descripcion_dataset(self):
    print(self.df.describe())

  def divide_train_test(self, variable, cantidad):

    self.train, self.test = self.df[[variable]].iloc[0:-cantidad], self.df[[variable]].iloc[-cantidad:len(self.df)]

    return self.train, self.test

  """
   Normalizar los data sets
  """
  def escala_train_test(self, data_train, data_test):
    self.scaler = StandardScaler()
    self.scaler.fit(data_train)  # Ajustar el scaler solo con los datos de entrenamiento
    train_set_scaled = self.scaler.transform(data_train)
    test_set_scaled = self.scaler.transform(data_test)  # Escalar los datos de prueba con los parámetros del scaler ajustados en los datos de entrenamiento
    # Convertir los arrays escalados de vuelta a DataFrames
    train_set_scaled_df = pd.DataFrame(train_set_scaled, columns=data_train.columns, index=data_train.index)
    test_set_scaled_df = pd.DataFrame(test_set_scaled, columns=data_test.columns, index=data_test.index)
    return train_set_scaled_df, test_set_scaled_df

  def valores_originales(self, y_test_esca, y_pred_esca, y_train_esca):
    y_test = self.scaler.inverse_transform(y_test_esca.reshape(-1, 1)).flatten()
    y_pred = self.scaler.inverse_transform(y_pred_esca.reshape(-1, 1)).flatten()
    y_train = self.scaler.inverse_transform(y_train_esca.reshape(-1, 1)).flatten()
    return y_test, y_pred, y_train
  """
    Define arquitectura
  """
  def create_dataset(self, X, y, v_time_steps):
    Xs, ys = [], []
    for i in range(len(X) - v_time_steps):
        v = X.iloc[i:(i + v_time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + v_time_steps])
    return np.array(Xs), np.array(ys)

  def crea_datasets_train_test(self, data_escalada, time_steps):
    X_data, y_data = self.create_dataset(data_escalada, data_escalada, time_steps)
    return  X_data, y_data

  def create_dataset_with_preds(self, X, y, pred_steps, v_time_steps):
    Xs, ys = [], []

    for i in range(len(X) - v_time_steps - pred_steps):
        v = X.iloc[i:(i + v_time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + v_time_steps + pred_steps - 1])

    return np.array(Xs), np.array(ys)

  def crea_datasets_train_test_preds(self, data_escalada, pred_steps, time_steps):
    X_data, y_data = self.create_dataset_with_preds(data_escalada, data_escalada, pred_steps, time_steps)
    X_data[:5]
    return  X_data, y_data



  def lstm_architecture(self, X_data, rate_dropout, units, num_layers):
    # Inicializando the RNN
    # self.model = Sequential()

    # Agregar capas LSTM y Dropout para regularización.
    for i in range(num_layers):
        # Si es la última capa, no se debe retornar secuencias
        return_sequences = True if i < num_layers - 1 else False
        self.model.add(LSTM(units=units, return_sequences=return_sequences, input_shape=(X_data.shape[1], X_data.shape[2])))
        self.model.add(Dropout(rate=rate_dropout))

    # Capa de Salida!
    self.model.add(Dense(units=1))

    # Resumen del modelo!
    self.model.summary()


  def compila_red_reuronal(self, X_train, rate_dropout, optimizador, perdida,units, num_layers):
    self.lstm_architecture(X_train,rate_dropout,units,num_layers)
    self.model.compile(optimizer = optimizador, loss = perdida)

  def ejecuta_red_reuronal(self, X_train, y_train, v_epochs, v_batch_size, v_shuffle):
    self.modelo_entrenado = self.model.fit(X_train,
                                          y_train,
                                          epochs=v_epochs,
                                          batch_size=v_batch_size,
                                          shuffle=v_shuffle)

  def grafica_de_perdida(self):
    plt.plot(self.modelo_entrenado.history['loss'], label='train')
    plt.legend()
    plt.show()

  def prediccion(self, X_test):

    y_pred = self.model.predict(X_test)
    y_pred=y_pred.reshape(-1, 1)

    return y_pred



  def grafica_predicciones(self, y_test, y_pred, y_train ):
    plt.figure(num=None, figsize=(15, 6), dpi=80, facecolor='w', edgecolor='k')
    #plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test.flatten(), marker='.', label="Valor real")
    plt.plot(np.arange(len(self.df['Total'])), self.df['Total'], marker='.', label="Valor real")
    plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred.flatten(), 'r', marker='.', label="Predicción")
    #plt.plot(np.arange(0, len(y_train)), y_train.flatten(), 'g', marker='.', label="Historial")
    plt.ylabel('')
    plt.xlabel('')
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(fontsize=20)
    plt.show()


  def grafica_proyeccion(self, y_test, y_pred, y_train ):
    plt.figure(num=None, figsize=(15, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred.flatten(), 'r', marker='.', label="Predicción")
    plt.plot(np.arange(0, len(y_train)), y_train.flatten(), marker='.', label="Historial")
    plt.ylabel('')
    plt.xlabel('')
    # Aumentar el tamaño de la fuente en los índices de los ejes x y y
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(fontsize=20)
    plt.show()


  def evaluacion_modelo(self, y_test, y_pred):
    # Vemos algunos indicadores del ajuste!
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f'RMSE: ',rmse)
    # Definimos y calculamos el MAPE (mean_absolute_percentage_error)
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f'MAPE: ',mape)
    return rmse, mape

  def dataframe_resultado(self,test, pred):
    self.df_resultado['Date']=self.df.index[-len(pred):]
    self.df_resultado['Valor_real']=test
    self.df_resultado['RNR LSTM']=pred

  def getdf_resultado(self):
    return self.df_resultado

In [ ]:
"""
   Variables:
   variable - Columna del dataset a usar en la serie de tiempo
   n_registros_test - número de registros contando desde el final de la serie a ser tomados como datos de test
   time_steps - lag de tiempo a predecir
   dropout - Porcentaje expresado en decimales para la cantidad de dropout de neuronas
   optimizador - metodo de optimización 'adam'
   perdida - metrica para controlar la perdida de gradiente
   n_epocs - cantidad de iteraciones
   batch  - tamaño del lote de cada iteracion
   v_shuffle  - si shuffle sera verdadero o falso

"""

def genera_modelo(variable, n_registros_test, time_steps, dropout, optimizador, perdida, n_epocs, batch, v_shuffle,n_units,n_num_layers):
  df_train, df_test = df.divide_train_test(variable, n_registros_test)
  df_train_escalado, df_test_escalado = df.escala_train_test(df_train, df_test)
  X_data_train, y_data_train = df.crea_datasets_train_test(df_train_escalado,time_steps)
  X_data_test, y_data_test = df.crea_datasets_train_test(df_test_escalado,time_steps)
  df.compila_red_reuronal(X_data_train, dropout, optimizador, perdida,n_units,n_num_layers)
  df.ejecuta_red_reuronal(X_data_train, y_data_train, n_epocs, batch, v_shuffle)
  df.grafica_de_perdida()
  y_predicho = df.prediccion(X_data_test)
  y_test_orig, y_pred_orig, y_train_orig = df.valores_originales(y_data_test, y_predicho, y_data_train)
  df.dataframe_resultado(y_test_orig,y_pred_orig)
  df.grafica_proyeccion(y_test_orig, y_pred_orig, y_train_orig)
  df.grafica_predicciones(y_test_orig, y_pred_orig, y_train_orig)
  print(y_predicho)
  print(len(y_predicho))

  return df.evaluacion_modelo(y_test_orig, y_pred_orig)


In [ ]:
"""
   Variables:
   variable - Columna del dataset a usar en la serie de tiempo
   n_registros_test - número de registros contando desde el final de la serie a ser tomados como datos de test
   time_steps - lag de tiempo a predecir
   dropout - Porcentaje expresado en decimales para la cantidad de dropout de neuronas
   optimizador - metodo de optimización 'adam'
   perdida - metrica para controlar la perdida de gradiente
   n_epocs - cantidad de iteraciones
   batch  - tamaño del lote de cada iteracion
   v_shuffle  - si shuffle sera verdadero o falso
   pred_steps - número de periodos a predecir

"""

def genera_modelo_con_preds(variable, n_registros_test, time_steps, dropout, optimizador, perdida, n_epocs, batch, v_shuffle,pred_steps,units,n_num_layers):

  df_train, df_test = df.divide_train_test(variable, n_registros_test)

  df_train_escalado, df_test_escalado = df.escala_train_test(df_train, df_test)
  X_data_train, y_data_train = df.crea_datasets_train_test_preds(df_train_escalado,pred_steps,time_steps)

  X_data_test, y_data_test = df.crea_datasets_train_test_preds(df_test_escalado,pred_steps,time_steps)

  df.compila_red_reuronal(X_data_train, dropout, optimizador, perdida,units,n_num_layers)
  df.ejecuta_red_reuronal(X_data_train, y_data_train, n_epocs, batch, v_shuffle)

  df.grafica_de_perdida()

  y_predicho = df.prediccion(X_data_test)

  y_test_orig, y_pred_orig, y_train_orig = df.valores_originales(y_data_test, y_predicho, y_data_train)

  df.grafica_predicciones(y_test_orig, y_pred_orig, y_train_orig)
  #pdb.set_trace()
  return df.evaluacion_modelo(y_test_orig, y_pred_orig)


